In [ ]:
import nltk 
from nltk.stem.lancaster import LancasterStemmer 
stemmer = LancasterStemmer()
from nltk.corpus import stopwords
import numpy as np 
import tflearn 
import tensorflow as tf
import random 
import json 

In [ ]:
# import our chat-bot intents file
import json
with open('../input/intents-lol/intents.json') as json_data:
    data = json.load(json_data)
print(data)

In [ ]:
words=[]
docs_x=[]
docs_y=[]
labels=[]

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds=nltk.word_tokenize(pattern)
        wrds=[stemmer.stem(w.lower()) for w in wrds if w not in ['?'] ]
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])
        if intent['tag'] not in labels :
            labels.append(intent['tag'])
print(docs_x)
print(docs_y)
        

In [ ]:
words=sorted(list(set(words)))
labels = sorted(labels)
print(words)
print(labels)
training = []
output =[] 
output_empty = [0 for _ in range ( len(labels))]

In [ ]:
# reset underlying graph data
bag_of_words=[]
print(docs_x)
print(words)

for docs in docs_x:
    bag=[]
    for w in words : 
        if w in docs :
            bag.append(1)
        else :
            bag.append(0)
    bag_of_words.append(bag)
train_x = np.array(bag_of_words)
print(train_x)

In [ ]:
output_y=[]
for docs in docs_y :
    output_bag=[]
    for label in labels :
        if docs == label :
            output_bag.append(1)
        else :
            output_bag.append(0)
    output_y.append(output_bag)

train_y=np.array(output_y)
print(train_y)

In [ ]:
# restore all of our data structures
tf.reset_default_graph()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

In [ ]:
def bag_of_words(s,words):
    bag=[0 for _ in range(len(words))]
    s_words=nltk.word_tokenize(s)
    s_words=[stemmer.stem(w.lower()) for w in s_words if w not in ['?']]
    for se in s_words :
        for i,w in enumerate(words):
            if se == w :
                bag[i]=1
    return np.array(bag)

In [ ]:
def chat():
    print("Start talking with the bot (Enter quit to exit): ")
    while True : 
        inp = input ("You : ")
        if inp.lower()== "quit" :
            break
        result = model.predict([bag_of_words(inp,words)])
        index = np.argmax(result)
        tag = labels [index]
        for tg in data['intents']:
            if tg['tag']== tag:
                responses = tg['responses']
        print(random.choice(responses))

In [ ]:
chat()